# Imports

In [ ]:
import pickle
import yaml
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

In [ ]:
with open('ros_packages/px4_ros_extended/src_py/params.yaml') as info:
    info_dict = yaml.load(info, Loader=yaml.SafeLoader)
        
n_test = info_dict["test_window_reward"]
n_train = info_dict["train_window_reward"]

# Dataframe Creation

In [ ]:
with open("logs/log_2.pkl", "rb") as file_pkl:
    data = pickle.load(file_pkl)
train = data[0]
test = data[1]

# Visualization

In [ ]:
# Visualize the result
means = pd.Series(train).rolling(n_train).mean()
stds = pd.Series(train).rolling(n_train).std()
x = range(0, len(means))
x_labels = range(0, len(means)*100, 100)
figure(figsize=(20, 8), dpi=80)
plt.plot(x, means, 'or')
plt.plot(x, means, '-', color='gray')

plt.fill_between(x, means - stds, means + stds,
                 color='gray', alpha=0.2)
plt.xlim(0, len(means)+1);
plt.xticks(ticks=x, labels=x_labels);

In [ ]:
means = pd.Series(test).rolling(n_test).mean()
stds = pd.Series(test).rolling(n_test).std()
x = range(0, len(means))
figure(figsize=(14, 8), dpi=80)
plt.plot(x, means, 'or')
plt.plot(x, means, '-', color='gray')

plt.fill_between(x, means - stds, means + stds,
                 color='gray', alpha=0.2)
plt.xlim(0, len(means)+1);